In [ ]:
!pip install -e $PWD/awscosts

In [ ]:
import pandas as pd
from datetime import datetime
from functools import reduce
from itertools import groupby, chain
from random import randint

import math

from iot.simulator import devices_generator, time_walker, resolution_period_finder, devices_date_stamper, devices_growth


## IoT requests generation with no growth

In [ ]:
device_number = 10000
request_period_in_seconds = 3600
start = datetime(2017, 1, 1)
end = datetime(2017, 1, 31)
resolution_in_seconds = 60

# Create device fleet generator (lazy generates a collection of time deltas)
devices = devices_generator(device_number, request_period_in_seconds)

# Create a generator for device periodic requests
request_period_generator = time_walker(start, request_period_in_seconds, end)

# Create a "stamper" that fixes deltas into actual datetime stamps for a specific period
stamper = devices_date_stamper(devices)

# Create a function that returns the t0 of a period for a given time
resolution_finder = resolution_period_finder(start, resolution_in_seconds)

# flatMap = map + reduce
all_requests = reduce(chain, map(stamper, request_period_generator), iter([]))

requests_by_resolution = map(lambda x: (x[0],len(list(x[1]))), groupby(all_requests, resolution_finder))


df = pd.DataFrame.from_records(list(requests_by_resolution), index='date', columns=['date', 'hits'])

In [ ]:
import awscosts

MB_per_request = 128
mylambda = awscosts.Lambda(MB_per_req=MB_per_request, ms_per_req=200)
myec2 = awscosts.EC2(instance_type='m4.4xlarge', MB_per_req=MB_per_request)

df['date'] = df.index
df['lambda_cost'] = df.apply(lambda x: mylambda.get_cost_per_hour(date = x['date'], reqs = x['hits']), axis=1)
df['ec2_cost'] = df.apply(lambda x: myec2.get_cost_per_hour(reqs = x['hits']), axis=1)
df['instances'] = df.apply(lambda x: myec2.get_num_instances(reqs = x['hits']), axis=1)


df.head()

# IoT requests generation with growth

## Linear growth

In [ ]:
def growth_function(current_devices, i):
    ni = int(math.floor(i * 0.01))
    return current_devices + [randint(0, request_period_in_seconds - 1) for _ in range(0, ni)], i + ni

devices = devices_generator(100, request_period_in_seconds)

devices_growth_generator = devices_growth(devices, growth_function)


stamper = devices_date_stamper(devices_growth_generator)
